In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import dash_html_components as html
import dash_core_components as dcc
import dash
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\Bram\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  """
C:\Users\Bram\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  


In [2]:
email_headers = pd.read_csv('data/email headers.csv', encoding='mac-roman')
email_headers['Date'] =  pd.to_datetime(email_headers['Date'], format='%m/%d/%Y %H:%M')

In [3]:
from_to = email_headers[['From', 'To', 'Date']]

def comma_cutter(sentence):
    names = []
    while sentence.find(',') != -1:
        index = sentence.find(',')
        names.append(sentence[:index])
        sentence = sentence[index + 1:]
    names.append(sentence)
    return names

from_new = []
to_new = []
time_new = []

for row in from_to.iterrows():
    to = comma_cutter(row[1][1])
    for name in to:
        if name[0] == ' ':
            name = name[1:]
        from_new.append(row[1][0])
        to_new.append(name)
        time_new.append(row[1][2])

In [4]:
names_dict = dict()
i = 0
for name in from_new:
    if name not in names_dict:
        names_dict[name] = i
        i += 1
for name in to_new:
    if name not in names_dict:
        names_dict[name] = i
        i += 1

In [5]:
def make_pca(hours):

    vector_list = []
    time_vector = []
    start_date = datetime.datetime(2014, 1, 7, 0, 0, 0)
    end_date = datetime.datetime(2014, 1, 19, 0, 0, 0)
    delta = datetime.timedelta(hours=hours)

    while start_date <= end_date:
        adjacency_mat = np.zeros((len(names_dict), len(names_dict)))
        for j in range(len(from_new)):
            if time_new[j] < start_date and time_new[j] > (start_date - delta):
                adjacency_mat[names_dict[from_new[j]]][names_dict[to_new[j]]] += 1
        vector_list.append(adjacency_mat.flatten())
        time_vector.append(start_date)
        start_date += delta

    df = pd.DataFrame(vector_list)
    return df, time_vector

In [6]:
def create_figure(hours):
    df, time_vector = make_pca(hours)
    x = df.values
    x = StandardScaler().fit_transform(x)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
                 , columns = ['principal component 1', 'principal component 2'])
    principalDf['Date'] = time_vector
    principalDf['Time'] = range(len(time_vector))
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', color = 'Time',
                     color_continuous_scale='Bluered_r', hover_data = ['Date'])
    return fig

In [7]:
app = dash.Dash(__name__)

In [8]:
app.layout = html.Div([
    html.H4('Pca representation of the email graphs'),
    dcc.Graph(
        id='pca'
    ),
    html.H4('Amount of hours per graph'),
    dcc.Slider(1, 24, 1,
               value=10,
               id='slider'
    ),
])

@app.callback(
    Output("pca", "figure"), 
     Input('slider', 'value')
)

def create_figure(slider):
    
    df, time_vector = make_pca(slider)
    
    
    x = df.values
    x = StandardScaler().fit_transform(x)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
                , columns = ['principal component 1', 'principal component 2'])
    principalDf['Date'] = time_vector
    principalDf['Time'] = range(len(time_vector))
    figure = px.scatter(principalDf, x='principal component 1', y='principal component 2', color = 'Time',
                     color_continuous_scale='Bluered_r', hover_data = ['Date'])

    return figure
    
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Apr/2022 16:27:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:24] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:24] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:24] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 16:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
